In [1]:
import cv2
import numpy as np
import pytesseract as pyt
pyt.pytesseract.tesseract_cmd = r'C:\Users\Thep Ho\AppData\Local\Programs\Tesseract-OCR\tesseract.exe'

In [2]:
def find_countours(img_sources):
    cnts = cv2.findContours(img_sources, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    return cnts

In [3]:
img = cv2.imread('images/root.png')
img_original = img.copy()
height, width, _ = img.shape

In [4]:
cv2.imshow("image root", img)
cv2.waitKey()

-1

In [5]:
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img_blur = cv2.GaussianBlur(img_gray, (5, 5), 0)

In [6]:
cv2.imshow("image prepro", img_blur)
cv2.waitKey()

-1

In [7]:
thresh = cv2.threshold(img_blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

In [8]:
cv2.imshow("thresh", thresh)
cv2.waitKey()

-1

In [9]:
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7, 1))
img_merge = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=5)

In [10]:
cv2.imshow("thresh merge", img_merge)
cv2.waitKey()

-1

In [11]:
img_edge = cv2.Canny(img_merge, 0, 255, 255)

In [12]:
cv2.imshow("thresh merge", img_edge)
cv2.waitKey()

-1

In [13]:
cnts = find_countours(img_edge)
cnts = sorted(cnts, key=cv2.contourArea, reverse=True)

In [14]:
process_area = height*width*(.75)

In [15]:
for cnt in cnts:
    if cv2.contourArea(cnt) < process_area:   
        x1, y1, x2, y2 = cv2.boundingRect(cnt)
        rod = 255 - thresh[y1:y1 + y2, x1:x1 + x2]
        cv2.rectangle(img_original, (x1, y1), (x1 + x2, y1 + y2), (0, 255, 0), 2)
        break
cv2.imwrite("output/rod.png", rod)

True

In [16]:
cv2.imshow("Roi image", rod)
cv2.imshow("origin image", img_original)
cv2.waitKey()

-1

In [17]:
image = cv2.imread('output/rod.png')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

In [18]:
average_area = 0
cnts = find_countours(thresh)
for c in cnts:
    x,y,width,height = cv2.boundingRect(c)
    average_area += width * height
average = average_area/len(cnts)

In [19]:
for c in cnts:
    x,y,width,height = cv2.boundingRect(c)
    area = width * height
    if area > average*3:  
        cv2.drawContours(thresh, [c], -1, (0,0,0), -1)

In [20]:
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7, 1))
dilate = cv2.dilate(thresh, kernel, iterations=5)

In [21]:
cnts = find_countours(dilate)
for c in cnts:
    area = cv2.contourArea(c)
    if area < average * 2:
        cv2.drawContours(thresh, [c], -1, (0,0,0), -1)

result = 255 - thresh

In [22]:
cv2.imshow('result', result)
cv2.waitKey()

-1

In [23]:
my_digits = pyt.image_to_string(result)
print("your digits is:", my_digits)

your digits is: 99951.

